In [44]:
import numpy as np
import pandas as pd
import os
import datetime as dt
import matplotlib.pyplot as plt

from pykrx import stock
from dateutil import relativedelta
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
def ch (name):
    cptype = name[7:8]
    maturity = name[9:15]
    strike = name[16:]

    return cptype+maturity+strike

def op_chain_df (cpflag, maturity, strike, data_list, s0=None):
    
    idx = (cpflag, str(maturity), str(strike))    
    df = data_list[idx].dropna()

    k = float(strike)

    if s0 is None:
        df['행사가격'] = [k for x in range(len(df))]
        return df

    else:
        df['행사가격'] = [k for x in range(len(df))]
        df['기초자산가'] = s0.loc[df.index]
        return df


def option_date_creator (week, date):
    '''마지막 만기 거래일이 나옴'''
    base_date = dt.datetime.strptime(date, '%Y-%m')
    option_date = base_date + relativedelta.relativedelta(weekday=3) + dt.timedelta(weeks=week)
    option_date = option_date.strftime('%Y-%m-%d')

    return option_date

# 데이터 불러오기

In [120]:
kospi200 = stock.get_index_ohlcv('20070801', '20221130', '1028')['종가']
file_path = os.getcwd() + '/Data/'

df_close   = pd.read_pickle(file_path+'Call_data_close.pkl')
df_base    = pd.read_pickle(file_path+'Call_data_base.pkl')
df_volume  = pd.read_pickle(file_path+'Call_data_volume.pkl')
df_imvol   = pd.read_pickle(file_path+'Call_data_imvol.pkl')
df_open    = pd.read_pickle(file_path+'Call_data_open.pkl')

df_close2   = pd.read_pickle(file_path+'Put_data_close.pkl')
df_base2    = pd.read_pickle(file_path+'Put_data_base.pkl')
df_volume2  = pd.read_pickle(file_path+'Put_data_volume.pkl')
df_imvol2   = pd.read_pickle(file_path+'Put_data_imvol.pkl')
df_open2    = pd.read_pickle(file_path+'Put_data_open.pkl')

data_list = [df_close, df_base, df_volume, df_imvol, df_open,\
             df_close2, df_base2, df_volume2, df_imvol2, df_open2]

test = []
ttt = ['종가', '기준가', '거래량', '내재변동성', '미결제약정', '종가', '기준가', '거래량', '내재변동성', '미결제약정']

for idx, dfdfdfdf in enumerate(data_list):

    tmp = dfdfdfdf.iloc[8:]
    tmp = tmp.set_axis(tmp.iloc[0], axis=1).drop(index=[8,9,10,11,12])

    tmp = tmp.rename(columns={tmp.columns[0] : 'Date'}).set_index('Date')
    tmp.index = pd.to_datetime(tmp.index)
    tmp.columns = [ch(x) for x in tmp.columns]

    k_list = [x[-5:] for x in tmp.columns]
    t_list = [x[1:-5] for x in tmp.columns]
    c_list = [x[0] for x in tmp.columns]
    f_list = [ttt[idx] for x in tmp.columns]
    ppp = pd.DataFrame(data=tmp.values, index=tmp.index, columns=[c_list, t_list, k_list, f_list])
    test.append(ppp)

df = pd.concat(test, axis=1)#, keys=ttt)
#df.columns = df.columns.swaplevel(0, 1).swaplevel(1,2).swaplevel(2,3)
df.columns.names = ['cpflag', 'tick', 'strike', 'feature']
df

FileNotFoundError: [Errno 2] No such file or directory: '/Users/Sun/Documents/Github/Derivatives-Trading-Strategy/Preprocessing/Data/Call_data_close.pkl'

In [118]:
df.columns.names = ['cpflag', 'tick', 'strike', 'feature']

## 데이터내부의 K의 Boundary와 만기일

In [8]:
ticker_list = [ch(x) for x in df_close.iloc[8].values][1:]
maturity_date = list(pd.Series([str(x[1:7]) for x in ticker_list]).unique())

upper = []
lower = []
k_list = []

for date in maturity_date:
    buffer = []
    for tick in ticker_list:
        if tick[1:7] == date:
            buffer.append(tick[7:])
        else:
            pass
    
    k_list.append(buffer)

    upper.append(buffer[np.argmax(buffer)])
    lower.append(buffer[np.argmin(buffer)])
    

reg = pd.DataFrame(data=[upper, lower], columns=maturity_date).T.rename(columns={0:'Upper', 1:'Lower'}).astype(float)
reg
reg['last_trade_date'] = [option_date_creator(1, f'{x[:4]}-{x[4:]}') for x in reg.index]
reg['k_list'] = k_list
reg

,Upper,Lower,last_trade_date,k_list
200709,270.0,165.0,2007-09-13,"[165.0, 167.5, 170.0, 172.5, 175.0, 177.5, 180..."
200710,275.0,192.5,2007-10-11,"[192.5, 195.0, 197.5, 200.0, 202.5, 205.0, 207..."
200711,277.5,192.5,2007-11-08,"[192.5, 195.0, 197.5, 200.0, 202.5, 205.0, 207..."
200712,277.5,195.0,2007-12-13,"[195.0, 197.5, 200.0, 202.5, 205.0, 207.5, 210..."
200801,277.5,210.0,2008-01-10,"[210.0, 212.5, 215.0, 217.5, 220.0, 222.5, 225..."
...,...,...,...,...
202207,515.0,222.5,2022-07-14,"[222.5, 225.0, 227.5, 230.0, 232.5, 235.0, 237..."
202208,490.0,222.5,2022-08-11,"[222.5, 225.0, 227.5, 230.0, 232.5, 235.0, 237..."
202209,520.0,220.0,2022-09-08,"[220.0, 222.5, 225.0, 227.5, 230.0, 232.5, 235..."
202210,480.0,180.0,2022-10-13,"[180.0, 182.5, 185.0, 187.5, 190.0, 192.5, 195..."


In [7]:
op_chain_df('P', 200712, 255.0, df, kospi200)

,종가,기준가,거래량,내재변동성,미결제약정,행사가격,기초자산가
Date,,,,,,,
2007-08-01,24.5,18.9,2,0.26,703,255.0,234.36
2007-08-02,25.75,24.5,24,0.28,719,255.0,233.84
2007-08-03,22.55,25.75,143,0.255,807,255.0,236.91
2007-08-06,24.6,22.55,0,0.472,807,255.0,233.92
2007-08-07,24.6,24.6,0,0.401,807,255.0,234.50
...,...,...,...,...,...,...,...
2007-12-07,8.55,6.75,16277,0.275,10087,255.0,247.38
2007-12-10,10.7,8.55,25972,0.24,9512,255.0,244.25
2007-12-11,7.35,10.7,13558,0.03,9339,255.0,246.85


## 근월물 원월물

In [98]:
times = df.index.strftime('%Y-%m-%d')
times = times[2578:]
times

Index(['2018-01-02', '2018-01-03', '2018-01-04', '2018-01-05', '2018-01-08',
       '2018-01-09', '2018-01-10', '2018-01-11', '2018-01-12', '2018-01-15',
       ...
       '2022-11-17', '2022-11-18', '2022-11-21', '2022-11-22', '2022-11-23',
       '2022-11-24', '2022-11-25', '2022-11-28', '2022-11-29', '2022-11-30'],
      dtype='object', name='Date', length=1211)

1달간 차이 보기

In [116]:
call_iv = df.loc['2018-01-02', ('C', '201802', slice(None), '내재변동성')]
put_iv  = df.loc['2018-01-02', ('P', '201802', slice(None), '내재변동성')]



C  201802  270.0  내재변동성      NaN
           272.5  내재변동성    0.107
           275.0  내재변동성    0.107
           277.5  내재변동성    0.107
           280.0  내재변동성    0.107
           282.5  내재변동성    0.107
           285.0  내재변동성    0.107
           287.5  내재변동성    0.107
           290.0  내재변동성    0.107
           292.5  내재변동성    0.107
           295.0  내재변동성    0.107
           297.5  내재변동성    0.107
           300.0  내재변동성     0.16
           302.5  내재변동성     0.17
           305.0  내재변동성     0.16
           307.5  내재변동성     0.15
           310.0  내재변동성    0.107
           312.5  내재변동성    0.145
           315.0  내재변동성     0.13
           317.5  내재변동성    0.125
           320.0  내재변동성     0.12
           322.5  내재변동성    0.113
           325.0  내재변동성    0.116
           327.5  내재변동성    0.113
           330.0  내재변동성    0.111
           332.5  내재변동성    0.108
           335.0  내재변동성    0.108
           337.5  내재변동성    0.106
           340.0  내재변동성    0.108
           342.5  내재변동성    0.107
          

In [97]:
df.[times[0], (:, , :, '내재변동성')]

SyntaxError: invalid syntax (798612580.py, line 1)

In [40]:
'''10월물 내재변동성'''
op10 = op_chain_df('C', 200710, 255.0, df, kospi200)[['내재변동성']]

'''9월물 내재변동성'''
op9 = op_chain_df('C', 200709, 255.0, df, kospi200)[['내재변동성']]


In [71]:
ticks = list(np.unique(np.array([x[1:7] for x in ticker_list])))
ticks = ticks[0:10]
for tick in zip(ticks[:-1], ticks[1:]):
    first = op_chain_df('C', tick[0], 255.0, df, kospi200)[['내재변동성']]
    next  = op_chain_df('C', tick[1], 255.0, df, kospi200)[['내재변동성']]
    
plt.figure(figsize=(12,6), facecolor='w')
plt.plot(op9,  label='200709')
plt.plot(op10, label='200710')

KeyError: ('C', '200804', '255.0')